In [1]:
import os
import pandas as pd
from tools.dataset import score_dataset
from tools.preprocessing.missing_values import get_missing_raio, delete_columns, impute_missing_values
from tools.preprocessing.outliers import delete_outliers, impute_outliers, get_limits

# 1. 데이터 로드

In [2]:
DATA_PATH = './data'

train = pd.read_csv(os.path.join(DATA_PATH, 'train.csv'))
test = pd.read_csv(os.path.join(DATA_PATH, 'test.csv'))

# train_origin = train.copy()
# test_origin = test.copy()

print(f'train shape: {train.shape}')
print(f'test shape: {test.shape}')

train shape: (79023, 76)
test shape: (24353, 75)


In [3]:
# 데이터 일부 확인
train.head()

,ID_LAT_LON_YEAR_WEEK,latitude,longitude,year,week_no,SulphurDioxide_SO2_column_number_density,SulphurDioxide_SO2_column_number_density_amf,SulphurDioxide_SO2_slant_column_number_density,SulphurDioxide_cloud_fraction,SulphurDioxide_sensor_azimuth_angle,...,Cloud_cloud_top_height,Cloud_cloud_base_pressure,Cloud_cloud_base_height,Cloud_cloud_optical_depth,Cloud_surface_albedo,Cloud_sensor_azimuth_angle,Cloud_sensor_zenith_angle,Cloud_solar_azimuth_angle,Cloud_solar_zenith_angle,emission
0,ID_-0.510_29.290_2019_00,-0.51,29.29,2019,0,-0.000108,0.603019,-0.000065,0.255668,-98.593887,...,3664.436218,61085.809570,2615.120483,15.568533,0.272292,-12.628986,35.632416,-138.786423,30.752140,3.750994
1,ID_-0.510_29.290_2019_01,-0.51,29.29,2019,1,0.000021,0.728214,0.000014,0.130988,16.592861,...,3651.190311,66969.478735,3174.572424,8.690601,0.256830,30.359375,39.557633,-145.183930,27.251779,4.025176
2,ID_-0.510_29.290_2019_02,-0.51,29.29,2019,2,0.000514,0.748199,0.000385,0.110018,72.795837,...,4216.986492,60068.894448,3516.282669,21.103410,0.251101,15.377883,30.401823,-142.519545,26.193296,4.231381
3,ID_-0.510_29.290_2019_03,-0.51,29.29,2019,3,NaN,NaN,NaN,NaN,NaN,...,5228.507736,51064.547339,4180.973322,15.386899,0.262043,-11.293399,24.380357,-132.665828,28.829155,4.305286
4,ID_-0.510_29.290_2019_04,-0.51,29.29,2019,4,-0.000079,0.676296,-0.000048,0.121164,4.121269,...,3980.598120,63751.125781,3355.710107,8.114694,0.235847,38.532263,37.392979,-141.509805,22.204612,4.347317


# 2. 결측값 처리

결측값 비율이 높은 칼럼 삭제

In [3]:
train_deleted, deleted_columns = delete_columns(train, 0.2, target='emission')

print(f'삭제된 칼럼 개수: {len(deleted_columns)}')

삭제된 칼럼 개수: 19


In [4]:
# 테스트 데이터에도 적용
test_deleted = test.drop(deleted_columns, axis=1)

In [5]:
# 결측값 제거 결과
data_list = [['train', train], ['train_deleted', train_deleted]]
results = []

for name, data in data_list:
    score = score_dataset(data, 'emission')
    results.append([name, score])

results

KeyboardInterrupt: 

칼럼을 삭제한 데이터셋의 스코어가 더 좋다.

가장 이상적인 결측값 대체법 찾기 

In [7]:
methods = ['mean', 'median', 'linear']
results = []

for method in methods:
    train_imputed = impute_missing_values(train_deleted, method)
    score = score_dataset(train_imputed, 'emission')
    results.append([method, score])
    print(f'method "{method}" 계산 완료')

results

AttributeError: 'tuple' object has no attribute 'drop'

'mean'으로 대체할 때 가장 좋은 score 기록

In [5]:
train_imputed, tool = impute_missing_values(train_deleted, 'mean')

test_imputed = test_deleted.fillna(tool)

In [23]:
score_dataset(train_imputed, 'emission')

8.75663387517149

# 3. 이상치 처리

numeric 칼럼에 대해 이상치 처리

In [6]:
outliers_columns = list(train_imputed.select_dtypes(exclude=['O']).columns)
outliers_columns.remove('emission')
outliers_columns[:10]

['latitude',
 'longitude',
 'year',
 'week_no',
 'SulphurDioxide_SO2_column_number_density',
 'SulphurDioxide_SO2_column_number_density_amf',
 'SulphurDioxide_SO2_slant_column_number_density',
 'SulphurDioxide_cloud_fraction',
 'SulphurDioxide_sensor_azimuth_angle',
 'SulphurDioxide_sensor_zenith_angle']

이상치 기준이 되는 상한, 하한 구하기

In [7]:
limits = get_limits(train_imputed, outliers_columns)
limits.items()

dict_items([('latitude', [0.41900000000000026, -4.173]), ('longitude', [32.2845, 27.448500000000003]), ('year', [2024.0, 2016.0]), ('week_no', [78.0, -26.0]), ('SulphurDioxide_SO2_column_number_density', [0.0003911345979763542, -0.00033896669728099033]), ('SulphurDioxide_SO2_column_number_density_amf', [1.1669939597287828, 0.4693074770428757]), ('SulphurDioxide_SO2_slant_column_number_density', [0.00031344770484945946, -0.0002734028406228577]), ('SulphurDioxide_cloud_fraction', [0.3137327257603556, 0.009091709129575035]), ('SulphurDioxide_sensor_azimuth_angle', [140.1330893219491, -150.97045989778013]), ('SulphurDioxide_sensor_zenith_angle', [65.45264699107074, 10.663640519012876]), ('SulphurDioxide_solar_azimuth_angle', [43.523391257981416, -208.53374808358552]), ('SulphurDioxide_solar_zenith_angle', [40.21798721033717, 15.809738889080622]), ('SulphurDioxide_SO2_column_number_density_15km', [0.0001327659731399024, -0.00011663270986185858]), ('CarbonMonoxide_CO_column_number_density', 

In [8]:
train_outliers_deleted, deleted_indices = delete_outliers(train_imputed, outliers_columns, limits)
train_impute_outliers = impute_outliers(train_imputed, outliers_columns, limits)

Index([], dtype='int64')
Index([], dtype='int64')
Index([], dtype='int64')
Index([], dtype='int64')
Index([    2,    21,    51,    55,    56,    67,    68,    81,    83,    86,
       ...
       78894, 78911, 78915, 78941, 78942, 78949, 78985, 78992, 79001, 79003],
      dtype='int64', length=6857)
Index([   48,   102,   158,   415,   454,   511,   570,   579,   597,   655,
       ...
       78937, 78983, 78986, 78987, 78988, 78989, 78990, 79015, 79016, 79018],
      dtype='int64', length=3799)
Index([    2,    21,    32,    51,    56,    67,    81,    83,    99,   102,
       ...
       78894, 78907, 78915, 78941, 78942, 78949, 78985, 78991, 78992, 79018],
      dtype='int64', length=6016)
Index([   14,   114,   116,   147,   148,   149,   228,   257,   272,   280,
       ...
       78823, 78835, 78857, 78858, 78882, 78915, 78952, 78964, 78997, 79022],
      dtype='int64', length=2344)
Index([27072], dtype='int64')
Index([   45,    47,    57,    81,   130,   157,   204,   257,   365, 

In [12]:
# 삭제된 행의 개수
len(train_outliers_deleted)

43388

이상치 처리 비교(이상치 처리 전: 8.75663387517149)

In [15]:
outliers_dfs = [['deleted', train_outliers_deleted], ['imputed', train_impute_outliers]]
results = []

for name, outliers_df in outliers_dfs:
    score = score_dataset(outliers_df, 'emission')
    results.append([name, score])

results

[['deleted', 8.49429524751788], ['imputed', 8.7388773949861]]

이상치가 있는 행을 제거했을 때가 가장 이상적인 score 기록

In [16]:
# 테스트 데이터의 이상치도 제거
test_outliers_deleted, deleted_indices = delete_outliers(test_imputed, outliers_columns, limits)

Index([], dtype='int64')
Index([], dtype='int64')
Index([], dtype='int64')
Index([], dtype='int64')
Index([    1,    12,    21,    22,    30,    41,    51,    56,    61,    79,
       ...
       24239, 24267, 24285, 24290, 24299, 24307, 24329, 24337, 24347, 24348],
      dtype='int64', length=1839)
Index([   16,    30,   116,   168,   215,   310,   315,   330,   332,   405,
       ...
       24150, 24161, 24176, 24207, 24225, 24231, 24253, 24264, 24305, 24323],
      dtype='int64', length=749)
Index([    1,    12,    21,    22,    41,    56,    61,    79,   105,   144,
       ...
       24180, 24207, 24210, 24212, 24223, 24228, 24267, 24290, 24299, 24348],
      dtype='int64', length=1397)
Index([    1,    12,    39,    42,    54,    60,    61,    82,    88,    91,
       ...
       24012, 24013, 24076, 24100, 24111, 24145, 24168, 24199, 24210, 24267],
      dtype='int64', length=1265)
Index([3852], dtype='int64')
Index([   17,   131,   164,   168,   180,   201,   218,   262,   315,   